In [18]:
#importing libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [19]:
#open the browser
driver=webdriver.Chrome()

#load the webpage
driver.get("https://www.redbus.in/online-booking/jkrtc")
time.sleep(3)
driver.maximize_window()
wait_time = WebDriverWait(driver, 20)

def get_jammukashmir_bus_data(xpath):
    bus_links = []  # Holds all the bus URLs
    bus_routes = []  # Holds all the route names
    
    for page_num in range(1, 3):
        route_elements = driver.find_elements(By.XPATH, xpath)
        bus_links.extend([element.get_attribute("href") for element in route_elements])
        bus_routes.extend([element.text for element in route_elements])
        
        try:
            pagination_table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//*[@class="DC_117_paginationTable"]'))
            )
            next_page_button = pagination_table.find_element(By.XPATH, f'//div[@class="DC_117_pageTabs " and text()="{page_num + 1}"]')
            
            # Scroll the next page button into view
            ActionChains(driver).move_to_element(next_page_button).perform()
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable(next_page_button))
            next_page_button.click()
            
        except NoSuchElementException:
            print(f"End of pages detected at page {page_num}")
            break
        except ElementClickInterceptedException:
            print(f"Could not click the next page button at page {page_num}, trying again.")
            time.sleep(1)  # Wait a moment before retrying
            next_page_button.click()
    
    return bus_links, bus_routes

In [20]:
# Run the function to get the bus links and routes
bus_links, bus_routes = get_jammukashmir_bus_data("//a[@class='route']")

# Now create the DataFrame with the returned variables
jammukashmir_df = pd.DataFrame({
    "Bus_Route_Name": bus_routes,  # variable name from ROUTE_TELANGANA
    "Bus_Route_Link": bus_links   # variable name from LINKS_TELANGANA
})

# Display the DataFrame
print(jammukashmir_df)


End of pages detected at page 2
                                  Bus_Route_Name  \
0                    Srinagar to Jammu (j and k)   
1                    Jammu (j and k) to Srinagar   
2                              Delhi to Srinagar   
3                       Jammu (j and k) to Delhi   
4                       Delhi to Jammu (j and k)   
5                    Jammu (j and k) to Amritsar   
6                    Jaipur (Rajasthan) to Delhi   
7                    Amritsar to Jammu (j and k)   
8                  Chandigarh to Jammu (j and k)   
9                  Jammu (j and k) to Chandigarh   
10                   Haridwar to Jammu (j and k)   
11  Katra (jammu and kashmir) to Jammu (j and k)   
12                   Jammu (j and k) to Haridwar   
13                     Poonch to Jammu (j and k)   
14            Jammu (j and k) to Mendhar (J & K)   
15                     Jammu (j and k) to Poonch   
16                   Jammu (j and k) to Kishtwar   
17                   Kishtwar to

In [25]:
# change dataframe to csv
path=r"C:\Users\Vanathi\Documents\redbus scraped details\jammukashmir_df.csv"
jammukashmir_df.to_csv(path,index=False)

In [26]:
df=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\jammukashmir_df.csv")

,Bus_Route_Name,Bus_Route_Link
0,Srinagar to Jammu (j and k),https://www.redbus.in/bus-tickets/srinagar-to-...
1,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...
2,Delhi to Srinagar,https://www.redbus.in/bus-tickets/delhi-to-sri...
3,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi
4,Delhi to Jammu (j and k),https://www.redbus.in/bus-tickets/delhi-to-jammu
5,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...
6,Jaipur (Rajasthan) to Delhi,https://www.redbus.in/bus-tickets/jaipur-to-delhi
7,Amritsar to Jammu (j and k),https://www.redbus.in/bus-tickets/amritsar-to-...
8,Chandigarh to Jammu (j and k),https://www.redbus.in/bus-tickets/chandigarh-t...
9,Jammu (j and k) to Chandigarh,https://www.redbus.in/bus-tickets/jammu-to-cha...


In [27]:
driver= webdriver.Chrome()
Busname_1 = []
Bustype_1 = []
Starttime_1 = []
Endtime_1 = []
Ratings_1 = []
Total_duration_1 = []
Prices_1 = []
Seats_available_1 = []
Route_names = []
Route_links = []

for i, r in df.iterrows():
    link = r["Bus_Route_Link"]
    routes = r["Bus_Route_Name"]

    # Open the link
    driver.get(link)
    time.sleep(2)

    # Click on elements to reveal bus details
    elements = driver.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")
    for element in elements:
        element.click()
        time.sleep(2)

    # Click elements to view bus details
    try:
        clicks = driver.find_element(By.XPATH, "//div[@class='button']")
        clicks.click()
    except:
        continue
    time.sleep(2)

    # Scroll through the page to load all bus details
    scrolling = True
    while scrolling:
        # Get the current scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        # Scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load
        time.sleep(5)

        # Get the new scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")

        # Check if the page height has changed
        if new_height == last_height:
            scrolling = False

    # Extract bus details
    try:
        bus_name = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='travels lh-24 f-bold d-color']"))
        )
        bus_type = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
        start_time = driver.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
        end_time = driver.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
        total_duration = driver.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
        try:
            rating = driver.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
        except:
            continue
        price = driver.find_elements(By.XPATH, '//*[@class="fare d-block"]')
        seats = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

        # Append data to respective lists
        for bus in bus_name:
            Busname_1.append(bus.text)
            Route_links.append(link)
            Route_names.append(routes)
        for bus_type_elem in bus_type:
            Bustype_1.append(bus_type_elem.text)
        for start_time_elem in start_time:
            Starttime_1.append(start_time_elem.text)
        for end_time_elem in end_time:
            Endtime_1.append(end_time_elem.text)
        for total_duration_elem in total_duration:
            Total_duration_1.append(total_duration_elem.text)
        for ratings in rating:
            Ratings_1.append(ratings.text)
        for price_elem in price:
            Prices_1.append(price_elem.text)
        for seats_elem in seats:
            Seats_available_1.append(seats_elem.text)
    except Exception as e:
        print(f"Error occurred while extracting details: {e}")
        continue

print("Successfully Completed")

Successfully Completed


In [28]:
# Before converting to DataFrame, check if all lists have the same length
lengths = {
    'Busname_1': len(Busname_1),
    'Bustype_1': len(Bustype_1),
    'Starttime_1': len(Starttime_1),
    'Endtime_1': len(Endtime_1),
    'Total_duration_1': len(Total_duration_1),
    'Prices_1': len(Prices_1),
    'Seats_available_1': len(Seats_available_1),
    'Route_names': len(Route_names),
    'Route_links': len(Route_links),
    'Ratings_1': len(Ratings_1)
}

print(lengths)

# Ensure that all lists have the same length
max_length = max(lengths.values())  # Find the longest list
for lst in [Busname_1, Bustype_1, Starttime_1, Endtime_1, Total_duration_1, Prices_1, Seats_available_1, Route_names, Route_links, Ratings_1]:
    while len(lst) < max_length:
        lst.append(None)  # Or you can append a placeholder like an empty string


{'Busname_1': 129, 'Bustype_1': 129, 'Starttime_1': 129, 'Endtime_1': 129, 'Total_duration_1': 129, 'Prices_1': 129, 'Seats_available_1': 129, 'Route_names': 129, 'Route_links': 129, 'Ratings_1': 0}


In [30]:
# Convert list to DataFrame
data = {
    'Bus_name': Busname_1,
    'Bus_type': Bustype_1,
    'Start_time': Starttime_1,
    'End_time': Endtime_1,
    'Total_duration': Total_duration_1,
    'Price': Prices_1,
    "Seats_Available": Seats_available_1,
    "Ratings": Ratings_1,
    'Route_link': Route_links,
    'Route_name': Route_names
}

df_buses_4= pd.DataFrame(data)

# Convert DataFrame to CSV
path = r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_4.csv"
df_buses_4.to_csv(path, index=False)

print("Successfully Completed and saved to CSV.")

Successfully Completed and saved to CSV.


In [31]:
d=pd.read_csv(r"C:\Users\Vanathi\Documents\redbus scraped details\df_buses_4.csv")
d

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_link,Route_name
0,JKRTC - 178988,NON A/C Seater (2+3),08:27,13:30,05h 03m,INR 289,42 Seats available,NaN,https://www.redbus.in/bus-tickets/amritsar-to-...,Amritsar to Jammu (j and k)
1,JKRTC - 178984,NON A/C Seater (2+3),11:00,16:30,05h 30m,INR 289,42 Seats available,NaN,https://www.redbus.in/bus-tickets/amritsar-to-...,Amritsar to Jammu (j and k)
2,JKRTC - 178985,NON A/C Seater (2+3),13:50,19:00,05h 10m,INR 289,42 Seats available,NaN,https://www.redbus.in/bus-tickets/amritsar-to-...,Amritsar to Jammu (j and k)
3,JKRTC - 178986,NON A/C Seater (2+3),15:20,20:30,05h 10m,INR 289,42 Seats available,NaN,https://www.redbus.in/bus-tickets/amritsar-to-...,Amritsar to Jammu (j and k)
4,JKRTC - 178987,NON A/C Seater (2+3),16:30,21:30,05h 00m,INR 289,42 Seats available,NaN,https://www.redbus.in/bus-tickets/amritsar-to-...,Amritsar to Jammu (j and k)
...,...,...,...,...,...,...,...,...,...,...
124,Lavish Travels,A/C Seater / Sleeper (2+2),01:00,08:15,07h 15m,INR 560,4 Seats available,NaN,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jammu (j and k)
125,SHAH HOLIDAY TRAVEL,"2+1, Ashok Leyland STILE Sleeper, AC",23:10,06:00,06h 50m,INR 799,26 Seats available,NaN,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jammu (j and k)
126,Perfect Holiday,A/C Sleeper (2+2),23:40,06:00,06h 20m,INR 999,24 Seats available,NaN,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jammu (j and k)
127,Pooja Travel Point,A/C Sleeper (2+1),03:05,09:40,06h 35m,INR 698,32 Seats available,NaN,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jammu (j and k)
